In [1]:
import sys

# Add your local path
sys.path.append('/workspaces/tubular')

In [2]:
from __future__ import annotations

from typing import TYPE_CHECKING

import narwhals as nw
import numpy as np
import pandas as pd
import polars as pl
from sklearn.cluster import KMeans

from tubular.base import BaseTransformer, DataFrameMethodTransformer
from tubular.mixins import (
    CheckNumericMixin,
    DropOriginalMixin,
    NewColumnNameMixin,
    TwoColumnMixin,
)

if TYPE_CHECKING:
    from narwhals.typing import (
        FrameT,
        IntoSeriesT,
    )

In [3]:

class BaseNumericTransformer(BaseTransformer, CheckNumericMixin):
    """
    Extends BaseTransformer for datetime scenarios.

    Parameters
    ----------
    columns : List[str]
        List of columns to be operated on.

    **kwargs
        Arbitrary keyword arguments passed onto BaseTransformer.init method.

    Attributes
    ----------
    columns : List[str]
        List of columns to be operated on

    polars_compatible : bool
        class attribute, indicates whether transformer has been converted to polars/pandas agnostic narwhals framework
    FITS: bool
        class attribute, indicates whether transform requires fit to be run first

    """

    polars_compatible = True

    FITS = False

    def __init__(self, columns: list[str], **kwargs: dict[str, bool]) -> None:
        super().__init__(columns=columns, **kwargs)

    @nw.narwhalify
    def fit(
        self,
        X: FrameT,
        y: nw.Series | None = None,
    ) -> BaseNumericTransformer:
        """Base fit method. Validates data and attributes prior to the child objects fit logic.

        Parameters
        ----------
        X : pd/pl.DataFrame
            A dataframe containing the required columns

        y : pd/pl.Series | None
            Required for pipeline.

        """

        super().fit(X, y)

        CheckNumericMixin.check_numeric_columns(self, X[self.columns])

        return self

    @nw.narwhalify
    def transform(self, X: FrameT) -> FrameT:
        """Base transform method. Validates data and attributes prior to the child objects tranform logic.

        Parameters
        ----------
        X : pd/pl.DataFrame
            Data to transform.

        Returns
        -------
        X : pd/pl.DataFrame
            Validated data

        """

        X = super().transform(X)

        CheckNumericMixin.check_numeric_columns(self, X[self.columns])

        return X


In [4]:
from sklearn import datasets

# Load the iris dataset
iris = datasets.load_iris()
df = iris.data
df = pl.DataFrame(df).select('column_0').rename({'column_0':'a'})
df = nw.from_native(df)

In [5]:
kmeans = KMeans(n_clusters=5)
kmeans.fit_predict(df)

/home/vscode/.local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 4, 4, 4, 1, 4, 1, 4, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 3, 0, 3, 4, 3, 4, 0, 1, 3, 1, 1, 0, 0, 0, 4, 3,
       4, 4, 0, 4, 0, 0, 0, 0, 0, 3, 3, 3, 0, 4, 4, 4, 4, 0, 4, 0, 3, 0,
       4, 4, 4, 0, 4, 1, 4, 4, 4, 0, 1, 4, 0, 4, 3, 0, 3, 2, 1, 2, 3, 2,
       3, 0, 3, 4, 4, 0, 3, 2, 2, 0, 3, 4, 2, 0, 3, 2, 0, 0, 0, 2, 2, 2,
       0, 0, 0, 2, 0, 0, 0, 3, 3, 3, 4, 3, 3, 3, 0, 3, 0, 0], dtype=int32)

In [6]:
groups = kmeans.predict(df)
groups = nw.from_native(pl.DataFrame(groups)).rename({'column_0':'groups'})
groups = groups.with_row_index()

In [7]:
df2 = nw.from_native(df)
df2 = df2.with_row_index()

In [8]:
results = df2.join(groups, on = 'index')
bins_max = results.group_by('groups').agg(
    nw.col('a').max()
    ).sort("a").select('a').to_numpy().ravel()

In [9]:
bins_max

array([5.2, 5.8, 6.4, 7.1, 7.9])

In [42]:
class OneDKmeansTransformer(BaseNumericTransformer):
    """Transformer that generates a new column based on kmeans algorithm.
    Transformer runs the kmean algorithm based on given number of clusters and then identifies the bins' cuts based on the results.
    Finally it passes them into the a cut function.
 
    Parameters
    ----------
    column : str
        Name of the column to discretise.
 
    new_column_name : str
        Name given to the new discrete column.
 
    n_clusters : int, default = 8
        The number of clusters to form as well as the number of centroids to generate.
 
    n_init "auto" or int, default="auto"
        Number of times the k-means algorithm is run with different centroid seeds.
        The final results is the best output of n_init consecutive runs in terms of inertia.
        Several runs are recommended for sparse high-dimensional problems (see `Clustering sparse data with k-means <https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#kmeans-sparse-high-dim>`__).
 
        When n_init='auto', the number of runs depends on the value of init: 10 if using init='random' or init is a callable;
        1 if using init='k-means++' or init is an array-like.
 
    kmeans_kwargs : dict, default = {}
        A dictionary of keyword arguments to be passed to the sklearn KMeans method when it is called in fit.
 
    **kwargs
        Arbitrary keyword arguments passed onto BaseTransformer.init().
 
    Attributes
    ----------
 
    polars_compatible : bool
        class attribute, indicates whether transformer has been converted to polars/pandas agnostic narwhals framework
    FITS: bool
        class attribute, indicates whether transform requires fit to be run first
 
    """
 
    polars_compatible = True
 
    FITS = True
 
    def __init__(
        self,
        column: str,
        new_column_name: str,
        n_init: str | int = "auto",
        n_clusters: int = 8,
        kmeans_kwargs: dict[str, object] | None = None,
        **kwargs: dict[str, bool],
    ) -> None:
        if not isinstance(new_column_name, str):
            msg = f"{self.classname()}: new_column_name should be a str but got type {type(new_column_name)}"
            raise TypeError(msg)
 
        if not isinstance(column, str):
            msg = f"{self.classname()}: column arg should be a single str giving the column to group."
            raise TypeError(msg)
 
        if not isinstance(n_clusters, int):
            msg = f"{self.classname()}: n_clusters should be a str but got type {type(n_clusters)}"
            raise TypeError(msg)
 
        if not (n_init == "auto" or isinstance(n_init, int)):
            msg = f"{self.classname()}: n_init should be 'auto' or int but got type {type(n_init)}"
            raise TypeError(msg)
 
        if kmeans_kwargs is None:
            kmeans_kwargs = {}
        else:
            if type(kmeans_kwargs) is not dict:
                msg = f"{self.classname()}: kmeans_kwargs should be a dict but got type {type(kmeans_kwargs)}"
                raise TypeError(msg)
 
        for i, k in enumerate(kmeans_kwargs.keys()):
            if type(k) is not str:
                msg = f"{self.classname()}: unexpected type ({type(k)}) for kmeans_kwargs key in position {i}, must be str"
                raise TypeError(msg)
 
        self.n_clusters = n_clusters
        self.new_column_name = new_column_name
        self.n_init = n_init
        self.kmeans_kwargs = kmeans_kwargs
 
        # This attribute is not for use in any method, use 'columns' instead.
        # Here only as a fix to allow string representation of transformer.
        self.column = column
 
        super().__init__(columns=[column], **kwargs)
 
    @nw.narwhalify
    def fit(self, X: FrameT, y: IntoSeriesT | None = None) -> OneDKmeansTransformer:
        """Fir transformer to input data.
 
        Parameters
        ----------
        X : pd/pl.DataFrame
            Dataframe with columns to learn scaling values from.
 
        y : None
            Required for pipeline.
 
        """
 
        super().fit(X, y)
 
        X = nw.from_native(X)
        if X.select(nw.col(self.column[0]).is_null().any()).to_numpy().ravel()[0]==True:
            msg = f"{self.classname()}: X should not contain missing values"
            raise TypeError(msg)

        kmeans = KMeans(
            n_clusters=self.n_clusters,
            n_init=self.n_init,
            **self.kmeans_kwargs,
        )
 
        native_namespace = nw.get_native_namespace(X).__name__
        groups = kmeans.fit_predict(X.select(self.columns[0]))
 
        X = X.with_columns(
            nw.new_series(
                name="groups",
                values=groups,
                backend=native_namespace,
            ),
        )
 
        self.bins = (
            X.group_by("groups")
            .agg(
                nw.col(self.columns[0]).max(),
            )
            .sort(self.columns[0])
            .select(self.columns[0])
            .to_numpy()
            .ravel()
        )
        return self
 
    @nw.narwhalify
    def transform(self, X: FrameT) -> FrameT:
        """Generate from input pd/pl.DataFrame (X) bins based on Kmeans results and add this column or columns in X.
 
        Parameters
        ----------
        X : pl/pd.DataFrame
            Data to transform.
 
        Returns
        -------
        X : pl/pd.DataFrame
            Input X with additional cluster column added.
        """
        X = super().transform(X)
 
        X = nw.from_native(X)
        native_namespace = nw.get_native_namespace(X).__name__
 
        groups = np.digitize(
            X.select(self.column[0]).to_numpy().ravel(),
            bins=self.bins,
            right=True,
        )
 
        return X.with_columns(
            nw.new_series(
                name=self.new_column_name,
                values=groups,
                backend=native_namespace,
            ),
        )
    
iris = datasets.load_iris()
df = iris.data
df = pl.DataFrame(df).select('column_0').rename({'column_0':'a'})
kmeans = OneDKmeansTransformer(column='a', new_column_name='new', ).fit(X=df)
df = kmeans.transform(df)

print(df)

df = iris.data
df = pd.DataFrame(df).rename(columns={0:'a'})
kmeans = OneDKmeansTransformer(column='a', new_column_name='new').fit(X=df)
df = kmeans.transform(df)

print(df)

shape: (150, 2)
┌─────┬─────┐
│ a   ┆ new │
│ --- ┆ --- │
│ f64 ┆ i64 │
╞═════╪═════╡
│ 5.1 ┆ 1   │
│ 4.9 ┆ 1   │
│ 4.7 ┆ 0   │
│ 4.6 ┆ 0   │
│ 5.0 ┆ 1   │
│ …   ┆ …   │
│ 6.7 ┆ 6   │
│ 6.3 ┆ 5   │
│ 6.5 ┆ 5   │
│ 6.2 ┆ 4   │
│ 5.9 ┆ 3   │
└─────┴─────┘
       a    1    2    3  new
0    5.1  3.5  1.4  0.2    1
1    4.9  3.0  1.4  0.2    1
2    4.7  3.2  1.3  0.2    0
3    4.6  3.1  1.5  0.2    0
4    5.0  3.6  1.4  0.2    1
..   ...  ...  ...  ...  ...
145  6.7  3.0  5.2  2.3    5
146  6.3  2.5  5.0  1.9    3
147  6.5  3.0  5.2  2.0    4
148  6.2  3.4  5.4  2.3    3
149  5.9  3.0  5.1  1.8    3

[150 rows x 5 columns]
